In [28]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

newnames_reco = {"aice_body-sharp":"DLR", 
                 "first_body":"MBIR", 
                 "fbp_fc08":"FBP", 
                 "aidr3d_fc08":"HIR", 
                 "asir_standard":"HIR", 
                 "fbp_standard":"FBP",
                 "qr40f-q3_0.4":"Qr40f-Q3_0.4",
                 "qr40f-q3_0.6":"Qr40f-Q3_0.6",
                 "qr40f-q3_1":"Qr40f-Q3_1",
                 "br40f-q3_0.4":"Br40f-Q3_0.4",
                 "br40f-q3_0.6":"Br40f-Q3_0.6",
                 "br40f-q3_1":"Br40f-Q3_1"}

newnames_ct = {
    "CT4": "<b>Vendor 1",
    "GE": "<b>Vendor 2",
    "photon":"<b>Vendor 3",
}

newnames_network = {
    "3d_fullres_LiTS_151":"<b>3D",
    "3d_fullres_LiTS_151_res":"<b>3Dres",
}

df_lesion_meta = pd.read_csv('results_csv/results_lesion_paper.csv')

dose_threshold = 12
measure_point_mapping = {
    # Canon
    '230915': '0',
    '231205': '0', 
    '250321': '1',
    '250328': '2',
    '250404': '3',
    '250411': '4',
    # GE
    '240306': '0',
    '250515': '1',
    '250618': '4',
}

size_filter =  4/3 * 3.14159265359 * (5/2)**3   # in cubic millimeters, 5mm sphere
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"volume_pred"] = None
df_lesion_meta.loc[(df_lesion_meta["volume_pred"] < size_filter) & (df_lesion_meta["volume_gt"].notna()),"dice"] = None
df_lesion_meta = df_lesion_meta[(df_lesion_meta["volume_pred"] > size_filter) | (df_lesion_meta["volume_pred"].isna()) ]

df_lesion_meta["measure_point"] = df_lesion_meta["date"].apply(lambda x: measure_point_mapping[str(x)] if str(x) in measure_point_mapping else None)
df_lesion_meta = df_lesion_meta[df_lesion_meta["CTDIvol"] < 36]

df_lesion_meta["detected"] = df_lesion_meta["dice"] > 0   
filter_lesion_props = (df_lesion_meta["C in HU"] < -10) | (df_lesion_meta["C in HU"].isna())
df_lesion_meta = df_lesion_meta[filter_lesion_props]
df_lesion_meta = df_lesion_meta[df_lesion_meta["kernel"] != "FC13"]

output_dir = "../../plots/3) deep learning performance eval/monitoring"

/var/folders/h7/vwbjm16917g8kt81x3my27d40000gn/T/ipykernel_71167/315103909.py:30: DtypeWarning:

Columns (1,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.



In [29]:
df_lesion_meta["lesion_prop"] = df_lesion_meta.apply(lambda x: f"{str(2*int(x['R in mm'])).zfill(2)}mm/{int(x['C in HU'])}HU" if not np.isnan(x['R in mm']) else None , axis=1)
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco"].str.lower() + "_" + df_lesion_meta["kernel"].str.lower()
df_lesion_meta["reco+kernel"] = df_lesion_meta["reco+kernel"].map(newnames_reco)
df_lesion_meta["network"] = df_lesion_meta["network"].map(newnames_network)
df_lesion_meta["ct"] = df_lesion_meta["ct"].map(newnames_ct)



In [30]:
df_lesions = pd.read_csv('results_csv/results_lesions_paper.csv')
df_lesions["reco+kernel"] = df_lesions["reco"].str.lower() + "_" + df_lesions["kernel"].str.lower()
df_lesions["reco+kernel"] = df_lesions["reco+kernel"].map(newnames_reco)
df_lesions["network"] = df_lesions["network"].map(newnames_network)
df_lesions["ct"] = df_lesions["ct"].map(newnames_ct)
df_lesions = df_lesions[df_lesions["kernel"] != "FC13"]
df_lesions["measure_point"] = df_lesions["date"].apply(lambda x: measure_point_mapping[str(x)] if str(x) in measure_point_mapping else None)


In [31]:
groupby_lesion_prop = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "measure_point"]
groupby_lesion_prop_rep = ["network", "CTDIvol", "reco+kernel", "ct", "lesion_prop", "R in mm", "C in HU", "rep", "measure_point"]
groupby_lesion_all = ["network", "CTDIvol", "reco+kernel", "ct", "measure_point"]
groupby_lesion_all_phantom_rep = ["network", "CTDIvol", "reco+kernel", "ct", "measure_point", "phantom", "rep"]
groupby_lesion_all_phantom = ["network", "CTDIvol", "reco+kernel", "ct", "measure_point", "phantom"]

In [32]:
df_lesion_recall_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_recall_rep = df_lesion_recall_rep.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_recall_rep.columns = groupby_lesion_all + ["rep"] + ["recall", "num_detected", "num_total"]

df_lesion_meta_precision_rep = df_lesion_meta.groupby(groupby_lesion_all + ["rep"]).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"]}).reset_index()
df_lesion_meta_precision_rep.columns = groupby_lesion_all + ["rep"] + ["precision", "num_detected", "num_total", "dice_mean", "dice_std"]

lesion_seg_summary_rep = df_lesion_recall_rep.merge(df_lesion_meta_precision_rep, on=groupby_lesion_all + ["rep"], how="left")
lesion_seg_summary_rep["F1-score"] = 2 * (lesion_seg_summary_rep["precision"] * lesion_seg_summary_rep["recall"]) / (lesion_seg_summary_rep["precision"] + lesion_seg_summary_rep["recall"])
lesion_seg_summary_rep["F1-score"] = lesion_seg_summary_rep["F1-score"].fillna(0)

In [33]:
df_lesion_prop_recall_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()] 
df_lesion_prop_recall_rep = df_lesion_prop_recall_rep.groupby(groupby_lesion_prop_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_prop_recall_rep.columns = groupby_lesion_prop_rep + ["recall", "num_detected", "num_total"]


In [34]:
lesions_seg_summary = df_lesions.groupby(groupby_lesion_all).agg({"dice":["mean", "std"]}).reset_index()
lesions_seg_summary.columns = groupby_lesion_all + ["dice_mean", "dice_std"]

In [35]:
df_lesion_meta_recall_phantom_rep = df_lesion_meta[~df_lesion_meta["filename_gt"].isna()]
df_lesion_meta_recall_phantom_rep = df_lesion_meta_recall_phantom_rep.groupby(groupby_lesion_all_phantom_rep).agg({"detected": ["mean", "sum", "count"]}).reset_index()
df_lesion_meta_recall_phantom_rep.columns = groupby_lesion_all_phantom_rep + ["recall", "num_detected", "num_total"]
df_lesion_meta_precision_phantom_rep = df_lesion_meta.groupby(groupby_lesion_all_phantom_rep).agg({"detected": ["mean", "sum", "count"], "dice":["mean", "std"]}).reset_index()
df_lesion_meta_precision_phantom_rep.columns = groupby_lesion_all_phantom_rep + ["precision", "num_detected", "num_total", "dice_mean", "dice_std"]
lesion_seg_summary_phantom_rep = df_lesion_meta_recall_phantom_rep.merge(df_lesion_meta_precision_phantom_rep, on=groupby_lesion_all_phantom_rep, how="left")
lesion_seg_summary_phantom_rep["F1-score"] = 2 * (lesion_seg_summary_phantom_rep["precision"] * lesion_seg_summary_phantom_rep["recall"]) / (lesion_seg_summary_phantom_rep["precision"] + lesion_seg_summary_phantom_rep["recall"])


In [36]:
lesion_seg_summary_rep["CTDIvol"] = lesion_seg_summary_rep["CTDIvol"].astype(float).round(5).astype(str)
lesions_seg_summary["CTDIvol"] = lesions_seg_summary["CTDIvol"].astype(float).round(5).astype(str)
lesion_seg_summary_phantom_rep["CTDIvol"] = lesion_seg_summary_phantom_rep["CTDIvol"].astype(float).round(5).astype(str)

In [37]:
# filter data for plots
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["num_total_x"] == 72]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["reco+kernel"] == "HIR"]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["measure_point"].isin(["0", "1", "4"])]
index_to_remove = (lesion_seg_summary_rep_plot["CTDIvol"] == "7.0") & (lesion_seg_summary_rep_plot["measure_point"] == "1") & (lesion_seg_summary_rep_plot["rep"] == 5)
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[~index_to_remove]

lesion_seg_summary_rep_plot["CTDIvol"] = lesion_seg_summary_rep_plot["CTDIvol"].astype(float)
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot.sort_values(by=["measure_point", "network", "CTDIvol" ,"rep"])
lesion_seg_summary_rep_plot["CTDIvol"] = lesion_seg_summary_rep_plot["CTDIvol"].astype(str)
# for each measure point, we want to know the difference in F1-score between this measure point and the first measure point
lesion_seg_summary_plot = lesion_seg_summary_rep_plot.groupby(["network", "CTDIvol", "reco+kernel", "measure_point"]).agg({"F1-score": "mean"}).reset_index()
lesion_seg_summary_plot["F1-score_diff"] = lesion_seg_summary_plot.groupby(["network", "CTDIvol", "reco+kernel"])["F1-score"].diff().fillna(0)

In [38]:
# save plot data for phillip jordan
lesion_seg_summary_rep_data = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_data = lesion_seg_summary_rep_data[lesion_seg_summary_rep_data["num_total_x"] == 72]
lesion_seg_summary_rep_data = lesion_seg_summary_rep_data[lesion_seg_summary_rep_data["reco+kernel"] == "HIR"]
lesion_seg_summary_rep_data = lesion_seg_summary_rep_data[lesion_seg_summary_rep_data["measure_point"].isin(["0", "1", "4"])]
index_to_remove = (lesion_seg_summary_rep_data["CTDIvol"] == "7.0") & (lesion_seg_summary_rep_data["measure_point"] == "1") & (lesion_seg_summary_rep_data["rep"] == 5)
lesion_seg_summary_rep_data = lesion_seg_summary_rep_data[~index_to_remove]

lesion_seg_summary_rep_data["CTDIvol"] = lesion_seg_summary_rep_data["CTDIvol"].astype(float)
lesion_seg_summary_rep_data = lesion_seg_summary_rep_data.sort_values(by=["measure_point", "network", "CTDIvol" ,"rep"])
lesion_seg_summary_rep_data["CTDIvol"] = lesion_seg_summary_rep_data["CTDIvol"].astype(str)


lesion_seg_summary_rep_data.to_csv(os.path.join("../../../../../../promotion/daten_philipp/Figure 4", "lesion_detection_monitoring_data.csv"), index=False)

In [39]:
# calc AUC for each curve F1 vs CTDIvol
from sklearn.metrics import auc

lesion_seg_summary_rep_auc = lesion_seg_summary_rep[(lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_auc = lesion_seg_summary_rep_auc[lesion_seg_summary_rep_auc["ct"].isin(["<b>Vendor 1", "<b>Vendor 2"])]
lesion_seg_summary_rep_auc = lesion_seg_summary_rep_auc[lesion_seg_summary_rep_auc["CTDIvol"].astype(float) < dose_threshold]

aucs = []
for group, df in lesion_seg_summary_rep_auc.groupby(["ct", "network", "reco+kernel", "rep", "measure_point"]):
    #print(group)
    x = df["CTDIvol"]
    #print(x)
    x = x.astype(float)
    x = x/x.max()
    y = df["F1-score"]
    #print(y)
    aucs.append((group, auc(x, y)))

aucs
aucs_df = pd.DataFrame(aucs, columns=["group", "AUC"])
# split group into network and reco+kernel and rep and keep AUC column
aucs_df["ct"] = aucs_df["group"].apply(lambda x: x[0])
aucs_df["network"] = aucs_df["group"].apply(lambda x: x[1])
aucs_df["reco+kernel"] = aucs_df["group"].apply(lambda x: x[2])
aucs_df["rep"] = aucs_df["group"].apply(lambda x: x[3])
aucs_df["measure_point"] = aucs_df["group"].apply(lambda x: x[4])
aucs_df = aucs_df.drop(columns=["group"])

aucs_df_mean = aucs_df.groupby(["ct","network", "reco+kernel", "measure_point"]).agg({"AUC":["mean", "std"]}).reset_index()
aucs_df_mean.columns = ["ct", "network", "reco+kernel", "measure_point", "AUC_mean", "AUC_std"]


In [40]:
fig = px.bar(aucs_df_mean[aucs_df_mean["measure_point"].isin(['0', '1', '4'])], x="measure_point", y="AUC_mean", error_y="AUC_std", color="reco+kernel", barmode="group",
       title="AUC for F1-score vs CTDIvol", template="simple_white", range_y=[0,1], facet_row="ct", facet_col="network",
       category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                        "ct": ["<b>Vendor 1", "<b>Vendor 2"],
                        "network": ["<b>3D", "<b>3Dres"]},
                        color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],)


fig.update_xaxes(title_text="<b>Reconstruction", row=1) 
fig.update_yaxes(title_text="<b>AUC", col=1)
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig.update_layout(
       font_family="Arial",
       font_size=14,
       title_font_family="Arial",
       title_font_color="black",
       height=500,
       width=600
       )
fig.show()
#fig.write_image(os.path.join(output_dir, f"lesion_seg_f1_score_auc_dicethreshold_{dice_detection_threshold}.png"))


In [41]:
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.multitest import multipletests

reco_comparisons = []
networks = []
vendors = []
reco_1_list = []
reco_2_list = []
measure_points_list = []
t_statistic = []
p_values = []
p_values_holm = []
p_values_othercorrection = []

for network in aucs_df["network"].unique():
    for ct in aucs_df["ct"].unique():
        reco_order = {"FBP": 0, "HIR": 1, "MBIR": 2, "DLR": 3}
        recos = aucs_df[aucs_df["network"] == network]["reco+kernel"].unique()
        recos = sorted(recos, key=lambda x: reco_order[x])
        measure_points = ['0', '1', '4']
        for measure_point in measure_points:
            p_values_temp = []
            for reco in recos:
                for reco_2 in recos[1:]:
                    if reco == "MBIR" or reco_2 == "MBIR":
                        continue
                    reco_comparison = f"{sorted([reco, reco_2])} for network {network} and ct {ct} and measure point {measure_point}"
                    if reco != reco_2 and reco_comparison not in reco_comparisons:
                        reco_comparisons.append(reco_comparison)
                        aucs_reco = aucs_df[(aucs_df["network"] == network) & (aucs_df["reco+kernel"] == reco) & (aucs_df["ct"] == ct) & (aucs_df["measure_point"] == measure_point)]["AUC"]
                        aucs_reco_2 = aucs_df[(aucs_df["network"] == network) & (aucs_df["reco+kernel"] == reco_2) & (aucs_df["ct"] == ct) & (aucs_df["measure_point"] == measure_point)]["AUC"]
                        if len(aucs_reco) == 0 or len(aucs_reco_2) == 0:
                            continue    
                        else:
                            print(f"{ct} {network} {reco} vs {reco_2} at measure point {measure_point}")
                        # print(f" AUCs {aucs_reco.values} vs {aucs_reco_2.values}")
                        t, p = ttest_rel(aucs_reco, aucs_reco_2)
                        p_values_temp.append(p)
                        networks.append(network)
                        vendors.append(ct)
                        reco_1_list.append(reco)
                        reco_2_list.append(reco_2)
                        t_statistic.append(t)
                        measure_points_list.append(measure_point)
            
            # replace p-values with nan 
            p_values_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_values_temp]
            p_values.extend(p_values_temp)

            # correct p-values for multiple comparisons
            reject, pvals_corrected, _, _ = multipletests(p_values_temp, method='holm')
            p_values_holm.extend(pvals_corrected)
            # you can add other correction methods if needed
            reject, pvals_corrected_other, _, _ = multipletests(p_values_temp, method='fdr_bh')
            p_values_othercorrection.extend(pvals_corrected_other)

df_stats_aucs_reco = pd.DataFrame({
    "network": networks,
    "ct": vendors,
    "reco_1": reco_1_list,
    "reco_2": reco_2_list,
    "t_statistic": t_statistic,
    "p_value": p_values,
    "p_value_holm": p_values_holm,
    "p_value_fdr_bh": p_values_othercorrection,
    "measure_point": measure_points_list
})

df_stats_aucs_reco.to_csv(os.path.join(output_dir, f"lesion_seg_f1_score_auc_stats_dicethreshold_{0}.csv"), index=False)

<b>Vendor 1 <b>3D FBP vs HIR at measure point 0
<b>Vendor 1 <b>3D FBP vs DLR at measure point 0
<b>Vendor 1 <b>3D HIR vs DLR at measure point 0
<b>Vendor 1 <b>3D FBP vs HIR at measure point 1
<b>Vendor 1 <b>3D FBP vs DLR at measure point 1
<b>Vendor 1 <b>3D HIR vs DLR at measure point 1
<b>Vendor 1 <b>3D FBP vs HIR at measure point 4
<b>Vendor 1 <b>3D FBP vs DLR at measure point 4
<b>Vendor 1 <b>3D HIR vs DLR at measure point 4
<b>Vendor 2 <b>3D FBP vs HIR at measure point 0
<b>Vendor 2 <b>3D FBP vs HIR at measure point 1
<b>Vendor 2 <b>3D FBP vs HIR at measure point 4
<b>Vendor 1 <b>3Dres FBP vs HIR at measure point 0
<b>Vendor 1 <b>3Dres FBP vs DLR at measure point 0
<b>Vendor 1 <b>3Dres HIR vs DLR at measure point 0
<b>Vendor 1 <b>3Dres FBP vs HIR at measure point 1
<b>Vendor 1 <b>3Dres FBP vs DLR at measure point 1
<b>Vendor 1 <b>3Dres HIR vs DLR at measure point 1
<b>Vendor 1 <b>3Dres FBP vs HIR at measure point 4
<b>Vendor 1 <b>3Dres FBP vs DLR at measure point 4
<b>Vendor 1 <b>3

In [43]:
from scipy.stats import ttest_rel, wilcoxon
from statsmodels.stats.multitest import multipletests

measure_point_comparisons = []
networks = []
vendors = []
mp_1_list = []
mp_2_list = []
t_statistic = []
p_values = []
p_values_holm = []
p_values_othercorrection = []

aucs_df_HIR = aucs_df[aucs_df["reco+kernel"] == "HIR"]

for network in aucs_df_HIR["network"].unique():
    for ct in aucs_df_HIR["ct"].unique():
        p_values_temp = []
        measure_points = ['0', '1', '4']
        for measure_point_1 in measure_points:
            for measure_point_2 in measure_points[1:]:
                measure_point_comparison = f"{sorted([measure_point_1, measure_point_2])} for network {network} and ct {ct}"
                if measure_point_1 != measure_point_2 and measure_point_comparison not in measure_point_comparisons:
                    measure_point_comparisons.append(measure_point_comparison)
                    aucs_meas_1 = aucs_df_HIR[(aucs_df_HIR["network"] == network) & (aucs_df_HIR["measure_point"] == measure_point_1) & (aucs_df_HIR["ct"] == ct)]["AUC"]
                    aucs_meas_2 = aucs_df_HIR[(aucs_df_HIR["network"] == network) & (aucs_df_HIR["measure_point"] == measure_point_2) & (aucs_df_HIR["ct"] == ct)]["AUC"]
                    if len(aucs_meas_1) == 0 or len(aucs_meas_2) == 0:
                        continue
                    else:
                        pass
                       # print(f"{ct} {network} {reco} vs {reco_2}")
                       # print(f" AUCs {aucs_reco.values} vs {aucs_reco_2.values}")
                    t, p = ttest_rel(aucs_meas_1, aucs_meas_2)
                    p_values_temp.append(p)
                    networks.append(network)
                    vendors.append(ct)
                    mp_1_list.append(measure_point_1)
                    mp_2_list.append(measure_point_2)
                    t_statistic.append(t)
        
        # replace p-values with nan 
        p_values_temp = [pv if not np.isnan(pv) else 1.0 for pv in p_values_temp]
        p_values.extend(p_values_temp)

        # correct p-values for multiple comparisons
        reject, pvals_corrected, _, _ = multipletests(p_values_temp, method='holm')
        p_values_holm.extend(pvals_corrected)
        # you can add other correction methods if needed
        reject, pvals_corrected_other, _, _ = multipletests(p_values_temp, method='fdr_bh')
        p_values_othercorrection.extend(pvals_corrected_other)

df_stats_aucs_monitoring = pd.DataFrame({
    "network": networks,
    "ct": vendors,
    "measure_point_1": mp_1_list,
    "measure_point_2": mp_2_list,
    "t_statistic": t_statistic,
    "p_value": p_values,
    "p_value_holm": p_values_holm,
    "p_value_fdr_bh": p_values_othercorrection
})

df_stats_aucs_monitoring.to_csv(os.path.join(output_dir, f"lesion_seg_f1_score_auc_stats_dicethreshold_HIR_{0}.csv"), index=False)

In [44]:
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep[(lesion_seg_summary_phantom_rep["ct"] == "<b>Vendor 1") & (lesion_seg_summary_phantom_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[lesion_seg_summary_phantom_rep_plot["num_total_x"] == 36]
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[lesion_seg_summary_phantom_rep_plot["reco+kernel"] == "HIR"]
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[lesion_seg_summary_phantom_rep_plot["measure_point"].isin(["0", "1", "4"])]
index_to_remove = (lesion_seg_summary_phantom_rep_plot["CTDIvol"] == "7.0") & (lesion_seg_summary_phantom_rep_plot["measure_point"] == "1") & (lesion_seg_summary_phantom_rep_plot["rep"] == 5)
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[~index_to_remove]


In [45]:
lesion_seg_summary_rep_plot["mean_F1_per_CTDI"] = lesion_seg_summary_rep_plot.groupby(["network", "CTDIvol"])["F1-score"].transform("mean")
lesion_seg_summary_rep_plot["F1-score_diff_per_CTDI"] = lesion_seg_summary_rep_plot["F1-score"] - lesion_seg_summary_rep_plot["mean_F1_per_CTDI"]

In [46]:
px.histogram(lesion_seg_summary_rep_plot, x="F1-score_diff_per_CTDI", color="network", barmode="overlay", nbins=100, range_x=[-0.2, 0.2], 
             title="Lesion segmentation F1-score distribution at CTDIvol 0.35 mGy (Vendor 1, HIR reconstruction)").show()

In [47]:
# First, create both figures separately
# Create strip plot (from cell 13)
fig_strip = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="measure_point", facet_col="network",
             title="Monitoring F1-score with difference overlay", range_y=[-0.1,1], template="simple_white", 
             hover_data=["precision", "recall","rep"],
             category_orders={"network": ["<b>3D", "<b>3Dres"]})


# Create a secondary y-axis in the first subplot
fig_strip.update_layout(
    yaxis2=dict(
        title="<b>F1-score Difference</b>",
        #titlefont=dict(color="gray"),
        tickfont=dict(color="gray"),
        overlaying="y",
        side="right",
        range=[-0.1, 1],  # Adjust this based on your data
        showgrid=False,
        visible=False,
    )
)

# Extract data from bar plot and add to strip plot with secondary axis
networks = ["<b>3D", "<b>3Dres"]
for network_idx, network in enumerate(networks):
    # Filter data for current network
    network_data = lesion_seg_summary_plot[lesion_seg_summary_plot["network"] == network]
    
    # For each measure point
    for measure_point in network_data["measure_point"].unique():
        mp_data = network_data[network_data["measure_point"] == measure_point]
        
        # Add bar for this network and measure point
        fig_strip.add_bar(
            x=mp_data["CTDIvol"], 
            y=mp_data["F1-score_diff"],
            marker=dict(
                opacity=1,
                color=fig_strip.data[2].marker.color if measure_point == "1" else 
                      fig_strip.data[4].marker.color if measure_point == "4" else "lightgray"
            ),
            name=f"{measure_point} diff",
            showlegend=False,
            yaxis="y2",
            xaxis=f"x{network_idx+1}",
            offsetgroup=measure_point
        )

# Update layout
fig_strip.update_xaxes(title_text="<b>CTDIvol", row=1)
fig_strip.update_yaxes(title_text="<b>F1-score", col=1)
fig_strip.update_xaxes(tickprefix="<b>", ticksuffix="</b>")
fig_strip.update_yaxes(tickprefix="<b>", ticksuffix="</b>")

fig_strip.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,  # Increased height to accommodate both visualizations
    width=1100,
    legend_title="<b>Measure Point</b>",
    barmode="group"
)

# Update annotations
fig_strip.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig_strip.show()
# Save the figure as an html file
fig_strip.write_html(os.path.join(output_dir, "lesion_segmentation_summary_canon.html"))
#fig_strip.write_image(os.path.join(output_dir, "lesion_segmentation_summary_canon.png"), width=1100, height=500)

In [48]:
# First, create both figures separately
# Create strip plot (from cell 13)
fig_strip = px.strip(lesion_seg_summary_phantom_rep_plot,
             x="CTDIvol", y="F1-score",
             color="measure_point", facet_col="network", facet_row="phantom",
             title="Monitoring F1-score with difference overlay", range_y=[-0.1,1], template="simple_white", 
             hover_data=["precision", "recall","rep"],
             category_orders={"network": ["<b>3D", "<b>3Dres"]})






# Update layout
fig_strip.update_xaxes(title_text="<b>CTDIvol", row=1)
fig_strip.update_yaxes(title_text="<b>F1-score", col=1)
fig_strip.update_xaxes(tickprefix="<b>", ticksuffix="</b>")
fig_strip.update_yaxes(tickprefix="<b>", ticksuffix="</b>")

fig_strip.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=800,  # Increased height to accommodate both visualizations
    width=1100,
    legend_title="<b>Measure Point</b>",
    barmode="group"
)

# Update annotations
fig_strip.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig_strip.show()
# Save the figure as an html file
fig_strip.write_html(os.path.join(output_dir, "lesion_segmentation_summary_canon_phantom.html"))


In [49]:
# # calculate AUC values for dose vs F1-score for lesion_seg_summary_rep_plot
# from sklearn.metrics import auc
# import matplotlib.pyplot as plt

# dose_threshold = 10
# # First, let's prepare the data by grouping and getting mean F1-scores per dose level
# auc_data = lesion_seg_summary_rep_data.groupby(["ct", "network", "measure_point", "CTDIvol"]).agg({
#     "F1-score": "mean"
# }).reset_index()

# auc_data = auc_data[auc_data["CTDIvol"].astype(float) < dose_threshold]

# # Convert CTDIvol to numeric for proper sorting
# auc_data["CTDIvol_numeric"] = auc_data["CTDIvol"].astype(float)
# auc_data = auc_data.sort_values(["network", "measure_point", "CTDIvol_numeric"])

# # Calculate AUC for each network and measure point combination
# auc_results = []

# for ct in auc_data["ct"].unique():
#     for network in auc_data["network"].unique():
#         for measure_point in auc_data["measure_point"].unique():
#             subset = auc_data[
#                 (auc_data["ct"] == ct) &
#                 (auc_data["network"] == network) &
#                 (auc_data["measure_point"] == measure_point)
#             ]

#             if len(subset) > 1:  # Need at least 2 points for AUC
#                 # Sort by dose to ensure proper order
#                 subset = subset.sort_values("CTDIvol_numeric")
                
#                 # Calculate AUC using trapezoidal rule
#                 x = subset["CTDIvol_numeric"].values
#                 y = subset["F1-score"].values
                
#                 # Normalize x to [0,1] for standardized AUC comparison
#                 x_norm = (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else x
                
#                 auc_value = auc(x_norm, y)
                
#                 auc_results.append({
#                     "ct": ct,
#                     "network": network,
#                     "measure_point": measure_point,
#                     "auc_normalized": auc_value,
#                     "auc_raw": auc(x, y),
#                     "dose_range": f"{x.min()}-{dose_threshold}",
#                     "f1_range": f"{y.min():.3f}-{y.max():.3f}",
#                     "n_points": len(subset)
#                 })

# # Convert to DataFrame for easier viewing
# auc_df = pd.DataFrame(auc_results)
# print("AUC Results (Dose vs F1-score):")
# print("="*50)
# print(auc_df.to_string(index=False))

# # Create a summary table
# print("\n\nAUC Summary Table:")
# print("="*30)
# auc_pivot = auc_df.pivot(index=["ct", "network"], columns="measure_point", values="auc_normalized")
# print(auc_pivot.round(4))

# # Optional: Visualize the dose-response curves with AUC values
# # fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# # networks = auc_data["network"].unique()


# # for i, network in enumerate(networks):
# #     network_data = auc_data[auc_data["network"] == network]
    
# #     for measure_point in network_data["measure_point"].unique():
# #         subset = network_data[network_data["measure_point"] == measure_point]
# #         subset = subset.sort_values("CTDIvol_numeric")
        
# #         # Get AUC value for annotation
# #         auc_val = auc_df[(auc_df["network"] == network) & 
# #                         (auc_df["measure_point"] == measure_point)]["auc_normalized"].iloc[0]
        
# #         axes[i].plot(subset["CTDIvol_numeric"], subset["F1-score"], 
# #                     marker='o', label=f'MP {measure_point} (AUC={auc_val:.3f})')
# #         axes[i].fill_between(subset["CTDIvol_numeric"], 0, subset["F1-score"], alpha=0.3)
    
# #     axes[i].set_xlabel('CTDIvol (mGy)')
# #     axes[i].set_ylabel('F1-score')
# #     axes[i].set_title(f'{network}')
# #     axes[i].legend()
# #     axes[i].grid(True, alpha=0.3)
# #     axes[i].set_ylim(0, 1)

# # plt.tight_layout()
# # plt.show()
# auc_df.to_csv("../../../../../../promotion/daten_philipp/Figure 4/lesion_detection_monitoring_data_auc.csv", index=False)


In [50]:
# # calculate AUC values for dose vs F1-score for lesion_seg_summary_phantom_rep
# from sklearn.metrics import auc
# import matplotlib.pyplot as plt

# # First, let's prepare the data by grouping and getting mean F1-scores per dose level
# auc_data = lesion_seg_summary_phantom_rep_plot.groupby(["ct","network", "measure_point", "CTDIvol", "phantom"]).agg({
#     "F1-score": "mean"
# }).reset_index()

# auc_data = auc_data[auc_data["CTDIvol"].astype(float) < dose_threshold]

# # Convert CTDIvol to numeric for proper sorting
# auc_data["CTDIvol_numeric"] = auc_data["CTDIvol"].astype(float)
# auc_data = auc_data.sort_values(["ct", "network", "measure_point", "CTDIvol_numeric" , "phantom"])

# # Calculate AUC for each network and measure point combination
# auc_results = []

# for ct in auc_data["ct"].unique():
#     for network in auc_data["network"].unique():
#         for measure_point in auc_data["measure_point"].unique():
#             for phantom in auc_data["phantom"].unique():
#                 subset = auc_data[
#                     (auc_data["ct"] == ct) &
#                     (auc_data["network"] == network) &
#                     (auc_data["measure_point"] == measure_point) &
#                              (auc_data["phantom"] == phantom)]

#             if len(subset) > 1:  # Need at least 2 points for AUC
#                 # Sort by dose to ensure proper order
#                 subset = subset.sort_values("CTDIvol_numeric")
                
#                 # Calculate AUC using trapezoidal rule
#                 x = subset["CTDIvol_numeric"].values
#                 y = subset["F1-score"].values
                
#                 # Normalize x to [0,1] for standardized AUC comparison
#                 x_norm = (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else x
                
#                 auc_value = auc(x_norm, y)
                
#                 auc_results.append({
#                     "ct": ct,
#                     "network": network,
#                     "phantom": phantom,
#                     "measure_point": measure_point,
#                     "auc_normalized": auc_value,
#                     "auc_raw": auc(x, y),
#                     "dose_range": f"{x.min()}-{x.max()}",
#                     "f1_range": f"{y.min():.3f}-{y.max():.3f}",
#                     "n_points": len(subset)
#                 })

# # Convert to DataFrame for easier viewing
# auc_phantom_df = pd.DataFrame(auc_results)
# print("AUC Results (Dose vs F1-score):")
# print("="*50)
# print(auc_phantom_df.to_string(index=False))

# # Create a summary table
# print("\n\nAUC Summary Table:")
# print("="*30)
# auc_pivot_phantom = auc_phantom_df.pivot(index=["ct", "network", "phantom"], columns="measure_point", values="auc_normalized")
# print(auc_pivot_phantom.round(4))



In [51]:
# # plot auc values using plotly
# fig = px.bar(auc_phantom_df, 
#              x="dose_range", 
#              y="auc_normalized", 
#              color="measure_point",
#              facet_col="network",
#              facet_row="phantom",
#              title="AUC Values: Dose vs F1-score Performance",
#              labels={
#                  "auc_normalized": "<b>Normalized AUC",
#                  "measure_point": "<b>Measure Point",
#                  "network": "<b>Network"
#              },
#              barmode="group",
#              hover_data=["auc_raw", "dose_range", "f1_range", "n_points"],
#              template="simple_white")

# # Update layout for better visualizationa
# fig.update_layout(
#     font_family="Arial",
#     font_size=14,
#     title_font_family="Arial",
#     title_font_color="black",
#     height=500,
#     width=1000,
#     showlegend=True,
#     legend_title="<b>Network</b>"
# )

# # Update axes
# fig.update_xaxes(title_text="")
# fig.update_yaxes(title_text="<b>Normalized AUC", range=[0, 1], col=1, row=1)
# fig.update_xaxes(tickprefix="<b>", ticksuffix="</b>")
# fig.update_yaxes(tickprefix="<b>", ticksuffix="</b>")

# # Remove facet column titles or update them
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))


# # Show the plot
# fig.show()

# #fig.write_image(os.path.join(output_dir, "auc_dose_vs_f1score_performance_canon.png"), width=1000, height=500)

In [57]:
dose_range = f"0-{min(lesion_seg_summary_rep_data['CTDIvol'].astype(float).max(), dose_threshold)}"
aucs_df_mean["dose_range"] = dose_range
# plot auc values using plotly
fig = px.bar(aucs_df_mean[ (aucs_df_mean["measure_point"].isin(['0', '1', '4'])) & (aucs_df_mean["reco+kernel"] == "HIR")],
             x="dose_range", 
             y="AUC_mean", 
             color="measure_point",
             facet_col="network",
             facet_row="ct",
             title="AUC Values: Dose vs F1-score Performance",
             labels={
                 "auc_normalized": "<b>Normalized AUC",
                 "measure_point": "<b>Measure Point",
                 "network": "<b>Network"
             },
             barmode="group",
             hover_data=["AUC_mean", "dose_range"],
             template="simple_white",
             category_orders={"ct": ["<b>Vendor 1", "<b>Vendor 2"],
                              "network": ["<b>3D", "<b>3Dres"]},)

# Update layout for better visualizationa
fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,
    width=1000,
    showlegend=True,
    legend_title="<b>Network</b>"
)

# Update axes
fig.update_xaxes(title_text="")
fig.update_yaxes(title_text="<b>Normalized AUC", range=[0, 1], col=1, row=1)
fig.update_xaxes(tickprefix="<b>", ticksuffix="</b>", matches=None)
fig.update_yaxes(tickprefix="<b>", ticksuffix="</b>")


# Remove facet column titles or update them
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))


# Show the plot
fig.show()

#fig.write_image(os.path.join(output_dir, "auc_dose_vs_f1score_performance_canon.png"), width=1000, height=500)

In [ ]:
lesion_seg_summary_rep_plot = lesion_seg_summary_rep[(lesion_seg_summary_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["num_total_x"] == 72]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["reco+kernel"] == "HIR"]
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[lesion_seg_summary_rep_plot["measure_point"].isin(["0", "1", "4"])]
index_to_remove = (lesion_seg_summary_rep_plot["CTDIvol"] == "7.0") & (lesion_seg_summary_rep_plot["measure_point"] == "1") & (lesion_seg_summary_rep_plot["rep"] == 5)
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[~index_to_remove]

lesion_seg_summary_rep_plot["CTDIvol"] = lesion_seg_summary_rep_plot["CTDIvol"].astype(float)
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot.sort_values(by=["measure_point", "network", "CTDIvol" ,"rep"])
lesion_seg_summary_rep_plot["CTDIvol"] = lesion_seg_summary_rep_plot["CTDIvol"].astype(str)
# for each measure point, we want to know the difference in F1-score between this measure point and the first measure point
lesion_seg_summary_plot = lesion_seg_summary_rep_plot.groupby(["network", "CTDIvol", "reco+kernel", "measure_point"]).agg({"F1-score": "mean"}).reset_index()
lesion_seg_summary_plot["F1-score_diff"] = lesion_seg_summary_plot.groupby(["network", "CTDIvol", "reco+kernel"])["F1-score"].diff().fillna(0)

In [ ]:
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep[(lesion_seg_summary_phantom_rep["ct"] == "<b>Vendor 2") & (lesion_seg_summary_phantom_rep["network"].isin(["<b>3D", "<b>3Dres"]))]
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[lesion_seg_summary_phantom_rep_plot["num_total_x"] == 36]
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[lesion_seg_summary_phantom_rep_plot["reco+kernel"] == "HIR"]
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[lesion_seg_summary_phantom_rep_plot["measure_point"].isin(["0", "1", "4"])]
index_to_remove = (lesion_seg_summary_phantom_rep_plot["CTDIvol"] == "7.0") & (lesion_seg_summary_phantom_rep_plot["measure_point"] == "1") & (lesion_seg_summary_phantom_rep_plot["rep"] == 5)
lesion_seg_summary_phantom_rep_plot = lesion_seg_summary_phantom_rep_plot[~index_to_remove]


In [ ]:
# remove fauty data week 4
lesion_seg_summary_rep_plot = lesion_seg_summary_rep_plot[~((lesion_seg_summary_rep_plot["CTDIvol"] == "22.4") & (lesion_seg_summary_rep_plot["measure_point"] == "4") & (lesion_seg_summary_rep_plot["rep"] == 5))]

In [ ]:
# First, create both figures separately
# Create strip plot (from cell 13)
fig_strip = px.strip(lesion_seg_summary_rep_plot,
             x="CTDIvol", y="F1-score",
             color="measure_point", facet_col="network",
             title="Monitoring F1-score with difference overlay", range_y=[-0.1,1], template="simple_white", 
             hover_data=["precision", "recall","rep"],
             category_orders={"network": ["<b>3D", "<b>3Dres"]})


# Create a secondary y-axis in the first subplot
fig_strip.update_layout(
    yaxis2=dict(
        title="<b>F1-score Difference</b>",
        #titlefont=dict(color="gray"),
        tickfont=dict(color="gray"),
        overlaying="y",
        side="right",
        range=[-0.1, 1],  # Adjust this based on your data
        showgrid=False,
        visible=False,
    )
)

# Extract data from bar plot and add to strip plot with secondary axis
networks = ["<b>3D", "<b>3Dres"]
for network_idx, network in enumerate(networks):
    # Filter data for current network
    network_data = lesion_seg_summary_plot[lesion_seg_summary_plot["network"] == network]
    
    # For each measure point
    for measure_point in network_data["measure_point"].unique():
        mp_data = network_data[network_data["measure_point"] == measure_point]
        
        # Add bar for this network and measure point
        fig_strip.add_bar(
            x=mp_data["CTDIvol"], 
            y=mp_data["F1-score_diff"],
            marker=dict(
                opacity=1,
                color=fig_strip.data[2].marker.color if measure_point == "1" else 
                      fig_strip.data[4].marker.color if measure_point == "4" else "lightgray"
            ),
            name=f"{measure_point} diff",
            showlegend=False,
            yaxis="y2",
            xaxis=f"x{network_idx+1}",
            offsetgroup=measure_point
        )

# Update layout
fig_strip.update_xaxes(title_text="<b>CTDIvol", row=1)
fig_strip.update_yaxes(title_text="<b>F1-score", col=1)
fig_strip.update_xaxes(tickprefix="<b>", ticksuffix="</b>")
fig_strip.update_yaxes(tickprefix="<b>", ticksuffix="</b>")

fig_strip.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=500,  # Increased height to accommodate both visualizations
    width=1100,
    legend_title="<b>Measure Point</b>",
    barmode="group"
)

# Update annotations
fig_strip.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig_strip.show()
# Save the figure as an html file
fig_strip.write_html(os.path.join(output_dir, "lesion_segmentation_summary_ge.html"))
#fig_strip.write_image(os.path.join(output_dir, "lesion_segmentation_summary_ge.png"), width=1100, height=500)

In [ ]:
# Create strip plot (from cell 13)
fig_strip = px.strip(lesion_seg_summary_phantom_rep_plot,
             x="CTDIvol", y="F1-score",
             color="measure_point", facet_col="network", facet_row="phantom",
             title="Monitoring F1-score with difference overlay", range_y=[-0.1,1], template="simple_white", 
             hover_data=["precision", "recall","rep"],
             category_orders={"network": ["<b>3D", "<b>3Dres"]})


# Update layout
fig_strip.update_xaxes(title_text="<b>CTDIvol", row=1)
fig_strip.update_yaxes(title_text="<b>F1-score", col=1)
fig_strip.update_xaxes(tickprefix="<b>", ticksuffix="</b>")
fig_strip.update_yaxes(tickprefix="<b>", ticksuffix="</b>")

fig_strip.update_layout(
    font_family="Arial",
    font_size=14,
    title_font_family="Arial",
    title_font_color="black",
    height=800,  # Increased height to accommodate both visualizations
    width=1100,
    legend_title="<b>Measure Point</b>",
    barmode="group"
)

# Update annotations
fig_strip.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))

fig_strip.show()
# Save the figure as an html file
fig_strip.write_html(os.path.join(output_dir, "lesion_segmentation_summary_ge_phantom.html"))


In [ ]:
#df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep[df_lesion_meta_recall_prop_rep["lesion_prop"] == "08mm/-40HU"]
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_prop_recall_rep[df_lesion_prop_recall_rep["ct"] == "<b>Vendor 1"] 
df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["network"].isin(["<b>3D"])]
#df_lesion_meta_recall_prop_rep_lesions_sub = df_lesion_meta_recall_prop_rep_lesions_sub[df_lesion_meta_recall_prop_rep_lesions_sub["rep"] == "rep1"]
df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"] = df_lesion_meta_recall_prop_rep_lesions_sub["CTDIvol"].astype(str)

fig = px.strip(df_lesion_meta_recall_prop_rep_lesions_sub,
             x="CTDIvol", y="recall",
             color="measure_point", facet_col="reco+kernel", facet_row="lesion_prop",
             title="recall", range_y=[0,1], template="simple_white", 
             hover_data=[ "recall"],
             color_discrete_sequence=["#2FA97C", "#0021E8", "#FF7F00", "darkred"],
             category_orders={"reco+kernel": ["FBP", "HIR", "MBIR", "DLR"],
                              "network": ["<b>3D"]})    

fig.update_layout(
    height=1500,
    width=1000
)

#fig.for_each_trace(lambda t: t.update({"marker":{"size":8}}))

fig.update_yaxes(title_text="<b>Recall", col=1)
fig.update_xaxes(title_text="<b>CTDIvol", row=1)

fig.update_yaxes(tickprefix="<b>",ticksuffix ="</b>")
fig.update_xaxes(tickprefix="<b>",ticksuffix ="</b>")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_annotation(lambda a: a.update(text=f"<b>{a.text}"))

#for row in range(1,3):
    #for col in range(1,7):
        #fig.add_shape(type='rect', x0=4, x1=16, y0=0, y1=1, line=dict(color="darkgreen", width=2),fillcolor="lightgreen", row=row, col=col)

fig.show()

